In [ ]:
from src.material.material.phong_material import *
from src import Sphere
from src import PointLight
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene
from src.render.loops import *
from src import Plane
from src.render.loops.progress import ProgressDisplay, PreviewConfig
from src.render.render_config import RenderConfig
from src import ipynb_display_images
from src.scene.surface_interaction import SurfaceInteraction
from src.shading import BlinnPhongShader, NormalShader, DotProductShader, DiffShader
from src.shading import HashMethod
from src.scene import Light
from src.shading.helpers import shadow_trace
from src.shading.shading_model import ShadingModel
from src import Color

In [ ]:
from src.scene.primitive import Object

glossy_brown = PhongMaterial(
    name="glossy_red",
    base_color=Color.custom_rgb(255, 100, 100),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=40,
    ior=1.5,
    reflectivity=0.4,
    transparency=0.0
)
point_light = PointLight(position=Vertex(5, 5, 0), intensity=1000.0, falloff=0.01)

camera = Camera(fov=50, aspect_ratio=Resolution.R360p.aspect_ratio, origin=Vertex(0, 0, 0), direction=Vector(0, 0, -1))

my_scene = Scene(camera=camera,
                 lights=[point_light],
                 skybox_path=None,
                 primitives=[
                     Object(
                         geometry=Plane(point=Vertex(0, -1, 0), normal=Vector(0, 1, 0)),
                         material=glossy_brown
                     ),
                     Object(
                         geometry=Sphere(center=Vertex(0.0, 0.0, -4.5), radius=1.0),
                         material=glossy_brown,
                     ),
                 ])

my_scene.validate()


In [ ]:
render_config = RenderConfig(
    resolution=Resolution.R360p,
    samples_per_pixel=20,
    max_depth=1,
)

preview_config = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

# Render with different shaders

## Blinn-Phong Shader
- Uses the Blinn-Phong shading model for rendering

In [ ]:
ray_tracer = LinearRayCaster(
    scene=my_scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=BlinnPhongShader(),
)

path = ray_tracer.render("images/shaders_blinn_phong.png")
ipynb_display_images(path)

## Normal Shader
- Shows the normal vectors as colors

In [ ]:
ray_tracer = MultiProcessRowRenderLoop(
    scene=my_scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=NormalShader(),
)

path = ray_tracer.render("images/shaders_normal_shader.png")
ipynb_display_images(path)

# Dot Product Shader (Light Direction and Surface Normal)
- Visualizes the dot product between light direction and surface normal

In [ ]:
ray_tracer = MultiProcessRowRenderLoop(
    scene=my_scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=DotProductShader(use_light=True)
)

path = ray_tracer.render("images/shaders_dot_product_shader_light.png")
ipynb_display_images(path)

# Dot Product Shader (Light Direction and View Direction)
- Visualizes the dot product between view direction and surface normal

In [ ]:
ray_tracer = MultiProcessRowRenderLoop(
    scene=my_scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=DotProductShader(use_light=False)
)

path = ray_tracer.render("images/shaders_dot_product_shader_view.png")
ipynb_display_images(path)

# Diff Shader
- show the difference between light direction and surface normal

In [ ]:
ray_tracer = MultiProcessRowRenderLoop(
    scene=my_scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=DiffShader(a=DotProductShader(use_light=True), b=NormalShader(), hash_method=HashMethod.HALF_IMAGE)
)

path = ray_tracer.render("images/shaders_diff_shader.png")
ipynb_display_images(path)

ray_tracer = MultiProcessRowRenderLoop(
    scene=my_scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=DiffShader(a=DotProductShader(use_light=True), b=NormalShader(), hash_method=HashMethod.STRIPES)
)
s 
path = ray_tracer.render("images/shaders_diff_shader.png")
ipynb_display_images(path)

# How to create a custom shading model
- For demonstration, we will create local shader \
that produces a repeating pattern based on distance

In [ ]:
class MyShadingModel(ShadingModel):
    def shade(self, hit, light, view_dir, scene=None):
        # Repeating pattern based on distance
        depth = hit.distance % 1.0
        col = 1.0 - abs(2.0 * depth - 1.0)
        return Color(col, col, col)

    def shade_multiple_lights(self, hit, lights, view_dir, scene=None):
        # Does not use lights
        return self.shade(hit, lights[0], view_dir, scene)

## Now we run it
- Using our custom local shading model in the ray tracer

In [ ]:
ray_tracer = LinearRayCaster(
    scene=my_scene,
    preview_config=preview_config,
    render_config=render_config,
    shading_model=MyShadingModel()
)

ray_tracer.render("images/shaders_custom_shader.png")